In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE161533"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE161533"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE161533.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE161533.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE161533.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# The dataset uses Affymetrix Gene Chip Human Genome U133 plus 2.0 Array, which contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: Examining the tissue type which indicates stomach cancer status
trait_row = 0  # 'tissue' field - has normal, paratumor, and tumor tissue types

# For age: Age data is available in key 2
age_row = 2  # 'age' field with multiple values

# For gender: Gender data is available in key 3
gender_row = 3  # 'gender' field with Male and Female values

# 2.2 Data Type Conversion

# Convert trait to binary (tumor vs non-tumor)
def convert_trait(value):
    if not isinstance(value, str):
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "tumor tissue" in value:
        return 1  # Tumor tissue (case)
    elif "normal tissue" in value:
        return 0  # Normal tissue (control)
    elif "paratumor tissue" in value:
        return None  # We'll exclude paratumor tissue as it's neither case nor control
    return None

# Convert age to continuous
def convert_age(value):
    if not isinstance(value, str):
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

# Convert gender to binary (0=female, 1=male)
def convert_gender(value):
    if not isinstance(value, str):
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "male" in value:
        return 1
    elif "female" in value:
        return 0
    return None

# 3. Save Metadata
# Initial validation - checking if gene and trait data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the function from the library
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers shown in the previous step
# These identifiers (e.g., '1007_s_at', '1053_at') appear to be Affymetrix probe IDs
# rather than standard human gene symbols (which would be like BRCA1, TP53, etc.)
# Affymetrix probe IDs need to be mapped to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns in the gene annotation dataframe
# The gene expression data uses identifiers like '1007_s_at', which matches the 'ID' column in the gene annotation
# The gene symbols are stored in the 'Gene Symbol' column

# 2. Create the gene mapping dataframe using the get_gene_mapping function
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print("\nGene mapping dataframe (first 5 rows):")
print(mapping_df.head())
print(f"Shape of mapping dataframe: {mapping_df.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("\nConverted gene expression data (first 5 genes, 5 samples):")
if not gene_data.empty:
    print(gene_data.iloc[:5, :5])
    print(f"\nShape of gene expression data: {gene_data.shape}")
else:
    print("Gene expression data is empty after mapping.")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in Step 2
# Inspect clinical data structure before manipulating
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(f"Loaded clinical data columns: {clinical_df.columns.tolist()}")

# Create clinical features DataFrame with proper structure 
# In this dataset, we know from step 2 that the first column is unnamed and contains feature names
clinical_features = clinical_df.copy()
# Set the first unnamed column as index if it exists
if 'Unnamed: 0' in clinical_features.columns:
    clinical_features = clinical_features.set_index('Unnamed: 0') 
clinical_features = clinical_features.T  # Transpose to get samples as rows and features as columns
print(f"Clinical features shape after format adjustment: {clinical_features.shape}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features.T, normalized_gene_data)
print(f"Linked data shape after linking: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from esophageal squamous cell carcinoma patients, with normal, paratumor, and tumor tissue samples."
)

# 6. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in Step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# Prepare clinical data properly, understanding the data structure from previous steps
# The DataFrame needs to be transposed to have samples as rows and features as columns
clinical_features = pd.DataFrame()
for col in clinical_df.columns:
    if col != 'Unnamed: 0':  # Skip the unnamed index column if it exists
        sample_id = col
        # Get trait, age, gender values for this sample
        values = clinical_df[col].values
        if len(values) >= 3:  # Make sure we have enough values
            clinical_features.loc[sample_id, trait] = values[0]  # Stomach_Cancer status
            clinical_features.loc[sample_id, 'Age'] = values[1]  # Age
            clinical_features.loc[sample_id, 'Gender'] = values[2]  # Gender

print(f"Prepared clinical features shape: {clinical_features.shape}")
print(clinical_features.head())

# Link clinical and genetic data
linked_data = pd.concat([clinical_features, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")
print(f"Linked data shape after removing biased features: {linked_data.shape}")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from esophageal squamous cell carcinoma patients, with normal, paratumor, and tumor tissue samples."
)

# 6. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")